# ipydatagrid

In [ ]:
from ipydatagrid import DataGrid, Filter, Sort, TextRenderer, BarRenderer, Expr, math, constants

In [ ]:
from json import load

with open('./cars.json') as fobj:
    data = load(fobj)

## Custom cell renderers

Custom cell renderers can be defined for the entire grid or column-wise.

Two types of cell renderers are currently available: `TextRenderer` and `BarRenderer`.

Most of the `TextRenderer`/`BarRenderer` attributes (`background_color`, `text_color` etc.) can either be a __value__, a __bqplot scale__ or a __`VegaExpr`__ or __`Expr`__ instance.


### Vega expressions
The `VegaExpr` class allows you to define an attribute value as a result of a Vega-expression (see https://vega.github.io/vega/docs/expressions/). _e.g._ `background_color = VegaExpr("value < 150 ? 'red' : 'green'")`.
  
You can look at the vega-expression documentation for more information about available constants and functions. In the scope of the expression are also available: `value`: cell value, `x` and `y`: cell position in pixel, `width` and `height` of the cell, `row` and `column`: cell position.

### Python expressions
If you prefer writing those expressions in Python, we provide an `Expr` class which takes a Python expression as input string or a Python function, and generates the equivalent vega-expression for you. Please keep in mind that only a subset of Python is supported. Only the `if` and `return` statements are supported, and assignment is prohibited.

Examples:
```Python
# Valid Python expression
def custom_background_color(value):
    if value < 100:
        return 'red'
    elif value < 150:
        return 'orange'
    else:
        return 'green'
    
background_color = Expr(custom_background_color)
```

```Python
# Invalid Python expression
def custom_background_color(value):
    test = 3  # Assignment is prohibited
    print('Hello')  # Function calls are prohibited
    # No return statement

background_color = Expr(custom_background_color)  # Will throw an error
```

In [ ]:
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale

def horsepower_renderer(value):
    if value < 100:
        return 'red'
    elif value < 150:
        return 'orange'
    else:
        return 'green'

renderers = {
    'Acceleration': BarRenderer(
        horizontal_alignment='center',
        bar_color=ColorScale(min=0, max=20, scheme='viridis'),
        value=LinearScale(min=0, max=20)
    ),
    'Cylinders': TextRenderer(
        background_color=Expr('"lightgrey" if row % 2 else "white"')
    ),
    'Displacement': TextRenderer(
        text_color=ColorScale(min=97, max=455), 
        font=Expr("'16px sans-serif' if value > 400 else '12px sans-serif'")
    ),
    'Horsepower': TextRenderer(
        background_color=Expr(horsepower_renderer)
    ),
    'Miles_per_Gallon': TextRenderer(
        background_color=Expr('"grey" if value is None else "white"')
    ),
    'Name': TextRenderer(
        background_color=Expr('"green" if "chevrolet" in value or "ford" in value else "white"')
    ),
    'Origin': TextRenderer(
        background_color=OrdinalColorScale(domain=['USA', 'Japan', 'Europe']),
        horizontal_alignment=Expr("'right' if value in ['USA', 'Japan'] else 'left'")
    ),
    'Year': TextRenderer(background_color='green'),
}

datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100, renderers=renderers)
datagrid

In [ ]:
renderers['Name'].background_color.value = '"green" if "pontiac" in value or "citroen" in value else "white"'

In [ ]:
datagrid.transform([Filter('Origin', '=', 'Europe'), Sort('Displacement')])

In [ ]:
datagrid.revert()

In [ ]:
datagrid.transform([Filter('Origin', '=', 'USA'), Filter('Acceleration', '<', 13), Sort('Acceleration')])

# Perfomance test: A million cells with ipydatagrid

In [ ]:
from random import uniform

def create_random_data(n_rows=100, n_columns=100):
    data = {'data': [], 'schema': {}}

    data['data'] = [[uniform(0, 1) for c in range(n_columns)] for r in range(n_rows)]
    data['schema']['fields'] = [{'name': str(c), type:'number'} for c in range(n_columns)]
    
    return data

In [ ]:
huge_data = create_random_data(1000, 1000)

def renderer_function():
    return '#fc8403' if value < constants.SQRT1_2 else 'white'

conditional_renderer = Expr(renderer_function)

conditional_huge_datagrid = DataGrid(data=huge_data, default_renderer=TextRenderer(background_color=conditional_renderer))
conditional_huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, Dropdown, ColorPicker, HBox

operator_dropdown = Dropdown(
    options=['<', '>'],
    value='<'
)
reference_slider = FloatSlider(value=0.5, min=0, max=1)
output_colorpicker = ColorPicker(
    value='#fc8403'
)

def on_change(*args, **kwargs):
    conditional_renderer.value = "'{color}' if value {operator} {reference} else 'white'".format(
        operator=operator_dropdown.value,
        reference=reference_slider.value,
        color=output_colorpicker.value
    )

operator_dropdown.observe(on_change, 'value')
reference_slider.observe(on_change, 'value')
output_colorpicker.observe(on_change, 'value')

HBox((operator_dropdown, reference_slider, output_colorpicker))

## Bar renderer

In [ ]:
from bqplot import LinearScale, ColorScale
from ipydatagrid import DataGrid, BarRenderer

linear_scale = LinearScale(min=0, max=1)
color_scale = ColorScale(min=0, max=1)
bar_renderer = BarRenderer(bar_color=color_scale, value=linear_scale, bar_horizontal_alignment='center')

huge_datagrid = DataGrid(data=create_random_data(), default_renderer=bar_renderer)

In [ ]:
huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(description='Scale: ', value=linear_scale.max, min=0, max=1)
link((color_scale, 'min'), (slider, 'value'))
link((linear_scale, 'min'), (slider, 'value'))

slider

In [ ]:
color_scale.scheme = 'magma'